In [ ]:
from pyomo.environ import *

from pyomo.opt import SolverFactory

import matplotlib.pyplot as plt

In [ ]:
# Definición del modelo
Model = ConcreteModel()

In [ ]:
# Conjunto de tareas (1 a 11)
T = RangeSet(1, 11)

# Constantes
NUM_DESARROLLADORES = 4
MAX_PUNTOS = 13

# Parámetros: puntos de historia por tarea
puntos_historia = {
    1: 5,
    2: 3,
    3: 13,
    4: 1,
    5: 21,
    6: 2,
    7: 2,
    8: 5,
    9: 8,
    10: 13,
    11: 21
}

# Parámetros: prioridades asignadas
prioridad = {
    "Maxima": 7,
    "Alta": 6,
    "Media Alta": 5,
    "Media": 4,
    "Media Baja": 3,
    "Baja": 2,
    "Minima": 1
}

# Mapeo de prioridades a tareas
prioridad_puntos = {
    1: "Maxima",
    2: "Media Alta",
    3: "Alta",
    4: "Media Baja",
    5: "Minima",
    6: "Media",
    7: "Alta",
    8: "Media",
    9: "Baja",
    10: "Maxima",
    11: "Alta"
}

# Conversión de prioridades a valores numéricos
num_prioridad_puntos = {t: prioridad[prioridad_puntos[t]] for t in prioridad_puntos}

## Variable de decision

In [ ]:
# Definición de las variables de decisión
Model.x = Var(T, domain=Binary)

## Modelo Matematico

**Conjuntos:**
$$T = \{T_1, T_2, \dots, T_{11}\}$$

**Parámetros:**
- **Prioridad de cada tarea:**
  $$P_i, \quad \forall i \in T$$
- **Puntos históricos de cada tarea:**
  $$H_i, \quad \forall i \in T$$
- **Total de puntos históricos:**
  $$H = \text{TotalHP}$$

**Variables de Decisión:**
$$X_i, \quad \forall i \in T$$

**Función Objetivo:**
$$\text{Maximizar} \sum_{i \in T} P_i \times X_i$$

**Restricciones:**
$$\sum_{i \in T} X_i \leq \text{TotalHP}$$


## Funcion objetivo

In [ ]:
# Definición de la función objetivo
Model.obj = Objective(expr=sum(Model.x[t] * num_prioridad_puntos[t] for t in T), sense=maximize)

## Restricciones

In [ ]:
# Restricción: la suma de los puntos de historia asignados no debe exceder la capacidad
Model.con1 = Constraint(expr=sum(Model.x[t] * puntos_historia[t] for t in T) <= MAX_PUNTOS * NUM_DESARROLLADORES)

## Solver

In [ ]:
# Resolución del modelo
SolverFactory('glpk').solve(Model)

## Model display

In [ ]:
# Mostrar resultados
Model.display()

## Grafica

In [ ]:
tasks = list(T)
story_points = [puntos_historia[t] for t in T]

selected = [Model.x[t]() for t in T] 


plt.bar(tasks, [sp * sel for sp, sel in zip(story_points, selected)], color='green', label='Selected Tasks')

plt.xlabel("Tareas")
plt.ylabel("Puntos de historia")
plt.title("Asignación de puntos de historia a tareas seleccionadas")

plt.xticks(tasks, [f"T{i}" for i in tasks])
plt.legend()

plt.show()

In [2]:
from IPython.display import Image, display

# URL of the image
image_url = "https://raw.githubusercontent.com/LuisCastelblanco/places/main/image.png"

# Display the image
display(Image(url=image_url))

## Discusion de resultados

### Distribución de Prioridades

Las tareas T1, T2, T3, T4, T6, T7, T8, T9, y T10 fueron seleccionadas, lo que refleja una diversidad en la distribución de prioridades. Las tareas con prioridad "Máxima" (T1 y T10) y "Alta" (T3, T7, y T11) fueron seleccionadas, lo que demuestra que el modelo dio preferencia a tareas críticas dentro de la limitación de puntos de historia disponible.

### Evaluacion de restricciones

La restricción de capacidad (13 puntos por desarrollador para 4 desarrolladores) fue respetada, asegurando que la carga de trabajo no excediera los recursos disponibles. El cuerpo de la restricción Body coincide con el límite superior 52 puntos, lo que indica una utilización completa de la capacidad.


### Implicaciones de decision
La no selección de tareas como T5 (prioridad "Mínima") y T11 (también "Alta") implica que estas tareas se postergaron posiblemente debido a sus elevados puntos de historia, lo que podría llevar a una acumulación de trabajo pendiente en futuras iteraciones.